In [ ]:
singer_name = 'test_kjh_diff' #@param {type: "string"}

In [ ]:
import re

%cd "diff-svc/"

os.environ['PYTHONPATH']='.'

!CUDA_VISIBLE_DEVICES=0

from utils.hparams import hparams
from preprocessing.data_gen_utils import get_pitch_parselmouth,get_pitch_crepe
import os
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import utils
import librosa
import torchcrepe
from infer import *
import logging
from infer_tools.infer_tool import *

print("Get Model")

logging.getLogger('numba').setLevel(logging.WARNING)

project_name = singer_name
# 모델 파일이 저장된 디렉토리 경로
model_dir = f'/app/{singer_name}/'

# 모든 파일 이름을 가져옵니다.
all_files = os.listdir(model_dir)

# 'model_ckpt_steps_x.ckpt' 형식의 파일만 걸러냅니다.
model_files = [f for f in all_files if re.match(r'model_ckpt_steps_\d+\.ckpt', f)]

# 파일 이름에서 숫자 부분만 추출하고 이를 정수로 변환합니다.
model_nums = [int(re.findall(r'\d+', f)[0]) for f in model_files]

# 가장 큰 숫자를 찾습니다.
latest_model_num = max(model_nums)

# 가장 큰 숫자를 가진 모델 파일의 이름을 만듭니다.
model_path = os.path.join(model_dir, f'model_ckpt_steps_{latest_model_num}.ckpt')
print(model_path)
config_path=f'/app/{singer_name}/config.yaml'
hubert_gpu=True
svc_model = Svc(project_name,config_path,hubert_gpu, model_path)
print('model loaded')

In [ ]:
import os

print("Output")

wav_in_folder = "/content/drive/MyDrive/test_kjh_diff" #@param {type: "string"}

# 모든 wav 파일 가져오기
wav_in_files = [os.path.join(wav_in_folder, file) for file in os.listdir(wav_in_folder) if file.endswith('.wav')]

# 출력 경로 설정
output_folder = f"/content/drive/MyDrive/{singer_name}/result"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)  # 경로가 존재하지 않는 경우 폴더 생성
key = 0


pndm_speedup = 100

add_noise_step = 1000

thre = 0.05
use_crepe= True
use_pe=False
use_gt_mel= True
# 각 입력 파일에 대해 처리
# 각 입력 파일에 대해 처리
for wav_in in wav_in_files:
    # 출력 파일 이름 생성 (입력 파일 이름에 프로젝트 이름 추가)
    wav_out_base = os.path.basename(wav_in)  # 입력 파일의 기본 이름 (확장자 포함)
    wav_out_base_no_ext = os.path.splitext(wav_out_base)[0]  # 확장자 제거
    wav_out = f"{wav_out_base_no_ext}_{singer_name}.wav"  # 프로젝트 이름 추가

    # 중복된 파일 이름이 존재할 경우 '_re' 추가
    while os.path.exists(f"{output_folder}/{wav_out}"):
        wav_out_base_no_ext = wav_out_base_no_ext + '_re'
        wav_out = f"{wav_out_base_no_ext}_{singer_name}.wav"

    # 모델 실행 및 결과 저장
    f0_tst, f0_pred, audio = run_clip(svc_model, file_path=wav_in, key=key, acc=pndm_speedup, use_crepe=use_crepe, use_pe=use_pe, thre=thre, use_gt_mel=use_gt_mel, add_noise_step=add_noise_step, project_name=project_name, out_path=f"{output_folder}/{wav_out}")
